In [1]:
#!/bin/bash
!kaggle datasets download salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:04<00:00, 240MB/s]
100% 1.06G/1.06G [00:04<00:00, 247MB/s]


In [2]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [17]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

In [18]:
data_norm = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1]
)

In [19]:
train_set = data_norm.flow_from_directory(
    "/content/dogs_vs_cats/train",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed=42
)

Found 20000 images belonging to 2 classes.


In [20]:
test_val = ImageDataGenerator(rescale=1.0 / 255)

test_set = test_val.flow_from_directory(
    "/content/dogs_vs_cats/test",
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False,
    seed=42
)

Found 5000 images belonging to 2 classes.


In [21]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

cnn_models = Sequential()

# Adding the first convolutional layer
cnn_models.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(128, 128, 3)))
cnn_models.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn_models.add(MaxPool2D(pool_size=2, strides=2))  # MaxPooling to reduce spatial dimensions

# Adding dropout to prevent overfitting
cnn_models.add(Dropout(0.25))

# Adding the second convolutional layer
cnn_models.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
cnn_models.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn_models.add(MaxPool2D(pool_size=2, strides=2))  # Another MaxPooling layer

# Adding another dropout layer
cnn_models.add(Dropout(0.25))

# Flatten the 2D feature maps into a 1D feature vector
cnn_models.add(Flatten())

# Check the output shape after flattening
cnn_models.summary()  # This will give you the shape before the Dense layer

# Fully connected (Dense) layers
cnn_models.add(Dense(units=512, activation='relu'))  # Dense layer with 512 neurons
cnn_models.add(Dense(units=256, activation='relu'))  # Dense layer with 256 neurons

# Adding another dropout layer for further regularization
cnn_models.add(Dropout(0.5))  # Dropout rate is higher here to avoid overfitting

# Output layer
cnn_models.add(Dense(units=36, activation='softmax'))  # Output layer with 36 classes

# Compile the model
cnn_models.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary to check the number of parameters and output shapes
cnn_models.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 126, 126, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 63, 63, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 61, 61, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 57600)               │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 65,568 (256.12 KB)

 Trainable params: 65,568 (256.12 KB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 126, 126, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 63, 63, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 61, 61, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │      29,491,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 36)                  │           9,252 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,697,860 (113.29 MB)

 Trainable params: 29,697,860 (113.29 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Compile the model using sparse categorical crossentropy
cnn_models.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = cnn_models.fit(
    train_set,
    validation_data=test_set,
    epochs=10,
    batch_size=32
)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 49s 70ms/step - accuracy: 0.5359 - loss: 0.7675 - val_accuracy: 0.6756 - val_loss: 0.5945
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.6872 - loss: 0.5875 - val_accuracy: 0.7440 - val_loss: 0.5133
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.7444 - loss: 0.5167 - val_accuracy: 0.7708 - val_loss: 0.4798
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 86s 73ms/step - accuracy: 0.7850 - loss: 0.4526 - val_accuracy: 0.7916 - val_loss: 0.4524
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 78s 66ms/step - accuracy: 0.8351 - loss: 0.3727 - val_accuracy: 0.8088 - val_loss: 0.4286
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 66ms/step - accuracy: 0.8764 - loss: 0.2919 - val_accuracy: 0.8098 - val_loss: 0.4501
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.9270 - loss: 0.1854 - val_accuracy: 0.7968 - val_loss: 0.5185
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 69ms/step - accuracy: 0.9580 - loss: 0.1108 - 

In [26]:
# Evaluate the model on the test set
test_loss, test_accuracy = cnn_models.evaluate(test_set)

# Print the test accuracy as a percentage
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss * 100:.2f}%")

157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 0.8014 - loss: 0.7568
Test Accuracy: 79.48%
Test Loss: 82.50%


In [25]:
cnn_models.save('cat_dog_classifier.h5')